# COSI 123a Term Project Final Model

In [5]:
import numpy as np
import pandas as pd
import csv
import os
from tqdm.auto import tqdm
from scipy.stats import rankdata
import Levenshtein
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load data

# Wild type sequence provided in the "Dataset Description"
wt = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

# Read testing set sequences and pH
test_df = pd.read_csv("models/input/novozymes-enzyme-stability-prediction/test.csv")

In [ ]:
# Add mutation information to testing set
result = []
for _, row in test_df.iterrows():
    ops = Levenshtein.editops(wt, row['protein_sequence'])
    assert len(ops) <= 1
    if len(ops) > 0 and ops[0][0] == 'replace':
        idx = ops[0][1]
        result.append(['SUB', idx + 1, wt[idx], row['protein_sequence'][idx]])
    elif len(ops) == 0:
        result.append(['WT', 0, '', ''])
    elif ops[0][0] == 'insert':
        assert False, "Ups"
    elif ops[0][0] == 'delete':
        idx = ops[0][1]
        result.append(['DEL', idx + 1, wt[idx], '_'])
    else:
        assert False, "Ups"

test_df = pd.concat([test_df, pd.DataFrame(data=result, columns=['type', 'resid', 'wt', 'mut'])], axis=1)

test_df

,seq_id,protein_sequence,pH,data_source,type,resid,wt,mut
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,17,L,E
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,17,L,K
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,DEL,17,L,_
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,18,K,C
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,18,K,F
...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,I
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,L
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,N
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,P


# Models

## Blosum

In [ ]:
def blosum_apply(row):
    if row['type'] == 'SUB':
        return blosum.loc[row['wt'], row['mut']]
    elif row['type'] == 'DEL':
        return -10
    elif row['type'] == 'WT':
        return 0
    else:
        assert False, "Ups"

blosum = pd.read_csv('models/input/blosum_data/BLOSUM100.txt', sep='\s+', comment='#')
test_df['blosum'] = test_df.apply(blosum_apply, axis=1)
test_df['blosum_rank'] = rankdata(test_df['blosum'])

test_df

,seq_id,protein_sequence,pH,data_source,type,resid,wt,mut,blosum,blosum_rank
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,17,L,E,-7,427.5
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,17,L,K,-6,659.0
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,DEL,17,L,_,-10,43.5
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,18,K,C,-8,207.5
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,18,K,F,-6,659.0
...,...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,I,-4,1316.0
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,L,-4,1316.0
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,N,-4,1316.0
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,P,-2,1838.5


## pLDDT

In [ ]:
plddt = (
    pd.read_csv('models/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb', sep='\s+', header=None)[[0,5,10]]
    .rename(columns={0:'atom', 5:'resid', 10:'plddt'})
    .query('atom=="ATOM"')
    .drop_duplicates()
)


# Add B factor to the testing set:
test_df = pd.merge(
    test_df,
    plddt,
    left_on='resid',
    right_on='resid',
    how='left'
)



test_df['plddt_rank'] = rankdata(-1*test_df['plddt'])

test_df

,seq_id,protein_sequence,pH,data_source,type,resid,wt,mut,blosum,blosum_rank,atom,plddt,plddt_rank
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,17,L,E,-7,427.5,ATOM,55.23,2408.0
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,17,L,K,-6,659.0,ATOM,55.23,2408.0
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,DEL,17,L,_,-10,43.5,ATOM,55.23,2408.0
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,18,K,C,-8,207.5,ATOM,69.25,2386.5
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,18,K,F,-6,659.0,ATOM,69.25,2386.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,I,-4,1316.0,ATOM,55.85,2398.5
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,L,-4,1316.0,ATOM,55.85,2398.5
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,N,-4,1316.0,ATOM,55.85,2398.5
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,SUB,16,A,P,-2,1838.5,ATOM,55.85,2398.5


## Differential pLDDT

In [11]:
plddtdiff = []

# Wild type result:
wt_plddt = (
    pd.read_csv('models/input/diff_pLDDT_data/nesp-kvigly-test-mutation-pdbs/WT_unrelaxed_rank_1_model_3.pdb', sep='\s+')
    .loc['ATOM'].reset_index()
    .loc[:, ['level_4', 'MODEL']].drop_duplicates()
    .rename(columns={'level_4':'resid', 'MODEL':'plddt'})
    .astype({'resid':int})
    .set_index('resid')
)

# Add difference in pLDDTto the testing set:>
for _,row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    file_path = 'models/input/diff_plddt_data/nesp-kvigly-test-mutation-pdbs/{}{}{}_unrelaxed_rank_1_model_3.pdb'.format(row['wt'], row['resid'], row['mut'])
    if os.path.exists(file_path):
        tdf = (
            pd.read_csv(file_path, sep='\s+')
            .loc['ATOM'].reset_index()
            .loc[:, ['level_4', 'MODEL']].drop_duplicates()
            .rename(columns={'level_4':'resid', 'MODEL':'plddt'})
            .astype({'resid':int})
            .set_index('resid')
        )
        plddtdiff.append((tdf.loc[row['resid']] - wt_plddt.loc[row['resid']]).values[0])
    else:
        plddtdiff.append(np.nan)

test_df['plddtdiff'] = plddtdiff
test_df['plddtdiff_rank'] = rankdata(test_df['plddtdiff'])

test_df

 60%|██████    | 1459/2413 [00:30<00:18, 52.63it/s]

## DeepDDG

In [ ]:
ddg = pd.read_csv('models/input/ddg_data/ddgout.csv')
print(ddg)

ddg_rank = ddg['ddg_rank']
test_df.merge(ddg_rank)

test_df


## DeMask

In [ ]:
demask = pd.read_csv('models/input/demask_data/demaskout.txt', sep='\t', usecols=[0,1,2,3], names=['resid','wt','mut','demask'], skiprows=1)
print(demask)

# Add DeMask output to the testing set:
test_df = pd.merge(
    test_df.set_index(['wt','resid','mut']),
    demask.set_index(['wt','resid','mut']),
    left_index=True,
    right_index=True,
    how='left'
).reset_index()

test_df.loc[test_df['type']=='WT','demask'] = 0
test_df.loc[test_df['type']=='DEL','demask'] = test_df['demask'].dropna().min()


test_df['demask_rank'] = rankdata(test_df['demask'])

test_df

## RMSD

In [ ]:
# Read VMD/NAMD output:
namd = pd.read_csv('models/input/RMSD_data/novozymes-md2/residue_rmsd_sasa_last.dat', sep='\t', header=None, names=['resid','rmsd','sasa0','sasaf'])

# Add VMD/NAMD results to the testing set:
test_df = pd.merge(
    test_df,
    namd[['resid','rmsd']],
    left_on='resid',
    right_on='resid',
    how='left'
)

test_df.loc[test_df['type']=='WT','rmsd'] = test_df['rmsd'].dropna().max()
# test_df.loc[test_df['type']=='WT','sasaf'] = test_df['sasaf'].dropna().max()

test_df['rmsd_rank'] = rankdata(test_df['rmsd'])

test_df

## SASA

In [ ]:
# Read VMD/NAMD output:
namd = pd.read_csv('models/input/SASA_data/novozymes-md/residue_rmsd_sasa_last.dat', sep='\t', header=None, names=['resid','rmsd','sasa0','sasaf'])

# Add VMD/NAMD results to the testing set:
test_df = pd.merge(
    test_df,
    namd[['resid','sasaf']],
    left_on='resid',
    right_on='resid',
    how='left'
)

# test_df.loc[test_df['type']=='WT','rmsd'] = test_df['rmsd'].dropna().max()
test_df.loc[test_df['type']=='WT','sasaf'] = test_df['sasaf'].dropna().max()
test_df['sasaf_rank'] = rankdata(test_df['sasaf'])

test_df

## Rosetta

In [ ]:
test_df['rosetta_rank'] = pd.read_csv('models/output/submission_rosetta_scores.csv')['tm']

test_df

## Thermonet

In [ ]:
test_df['thermonet'] = pd.read_csv('models/output/thermonet_data/thermonet_submission.csv')['tm']
test_df['thermonet_rank'] = rankdata(test_df['thermonet'])

## Export Model Results Before Ensembling

In [ ]:
print('Final test dataframe with rank scores:')
print(test_df.head())
test_df.to_csv('ensemble.csv', index=False)

# Ensemble

In [ ]:
def rank_nrom(name):
    s = test_df['{}_rank'.format(name)]
    return s/s.max()

In [ ]:
# Ensemble insertion and substitution type mutations
test_df['tm'] = (
    5*rank_nrom('rosetta') + 3*rank_nrom('rmsd') + 3*rank_nrom('thermonet') + 3*rank_nrom('plddtdiff') +\
    rank_nrom('sasaf') + rank_nrom('plddt') + rank_nrom('demask') + rank_nrom('ddg') + rank_nrom('blosum')
)

# Normalize results
test_df['tm'] = test_df['tm']/test_df['tm'].max()

# Ensemble deletion type mutations
idx = test_df[test_df['type']=="DEL"].index
test_df.loc[idx, 'tm'] = (
    2*rank_nrom('plddt')[idx] + 2*rank_nrom('plddtdiff')[idx] + rank_nrom('rmsd')[idx] + rank_nrom('sasaf')[idx]
) / 6

# Ensemble wildtype mutation
test_df.loc[test_df['type']=="WT",'tm'] = test_df['tm'].max()+1

# Rank data
test_df['tm'] = rankdata(test_df['tm'])
test_df['tm_rank'] = test_df['tm']

# Save final submission
test_df[['seq_id','tm']].to_csv('submission_final.csv', index=False)